In [1]:
#! sudo pip install --upgrade pip
#! sudo pip install meshio
#! sudo pip install h5py

In [2]:
from fenics import *
from fenics_adjoint import *

In [3]:
mesh = Mesh("test_mesh_small.xml")
target_space  = VectorFunctionSpace(mesh, 'DG', 2, 5)
control_space = TensorFunctionSpace(mesh, 'DG', 2, (5, 5))
base_space    = FunctionSpace(mesh, 'DG', 2)

In [4]:
partition = MeshFunction('int', mesh, 'test_mesh_small_cell_tags.xml')
subdomains = MeshFunction('size_t', mesh, mesh.topology().dim())
for c in cells(mesh):
    if partition[c] == -6: subdomains[c] = 0
    if partition[c] == -7: subdomains[c] = 1

In [5]:
# parameters from diploma

pspeed_0 = 1600.
sspeed_0 = 800.
rho_0    = 1100.

pspeed_1 = 4000.
sspeed_1 = 2000.
rho_1    = 1800.

def calculate_lame_params(pspeed, sspeed, rho):
    return (pspeed ** 2 - 2. * sspeed ** 2) * rho, sspeed ** 2 * rho 

lambda_0, mu_0 = calculate_lame_params(pspeed_0, sspeed_0, rho_0)
lambda_1, mu_1 = calculate_lame_params(pspeed_1, sspeed_1, rho_1)

In [6]:
class lambda_(UserExpression):
    
    def __init__(self, materials, lambda_0, lambda_1, **kwargs):
        
        super().__init__(**kwargs)
        
        self.materials = materials
        self.lambda_0  = lambda_0
        self.lambda_1  = lambda_1
        
    def eval_cell(self, values, x, cell):
        
        if self.materials[cell.index] == 0:
            values[0] = self.lambda_0
        else:
            values[0] = self.lambda_1
            
        return values
            
class mu_(UserExpression):
    
    def __init__(self, materials, mu_0, mu_1, **kwargs):
        
        super().__init__(**kwargs)
        
        self.materials = materials
        self.mu_0  = mu_0
        self.mu_1  = mu_1
        
    def eval_cell(self, values, x, cell):
        
        if self.materials[cell.index] == 0:
            values[0] = self.mu_0
        else:
            values[0] = self.mu_1
            
        return values

            
class rho_(UserExpression):
    
    def __init__(self, materials, rho_0, rho_1, **kwargs):
        
        super().__init__(**kwargs)
        
        self.materials = materials
        self.rho_0  = rho_0
        self.rho_1  = rho_1
        
    def eval_cell(self, values, x, cell):
        
        if self.materials[cell.index] == 0:
            values[0] = self.rho_0
        else:
            values[0] = self.rho_1
            
        return values
            
        
ctrls_mu      = Function(base_space)
ctrls_lambda  = Function(base_space)
ctrls_rho     = Function(base_space)

        
# set constant values

ctrls_lambda.assign(
    interpolate(lambda_(subdomains, lambda_0, lambda_1, degree=2), base_space))
ctrls_mu.assign(
    interpolate(mu_(subdomains, mu_0, mu_1, degree=2), base_space))
ctrls_rho.assign(
    interpolate(rho_(subdomains, rho_0, rho_1, degree=2), base_space))

In [7]:
# assemble matrix A as a tensor-valued function

A = Function(control_space)
assigner_A = FunctionAssigner(
    control_space, [base_space] * control_space.num_sub_spaces())

comps_A = [
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_lambda - 2 * ctrls_mu, base_space),
        interpolate(Constant(0.0), base_space)
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_lambda, base_space),
        interpolate(Constant(0.0), base_space),
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_mu, base_space)
    ],
    *[
        project(-1. / ctrls_rho, base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space)
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-1. / ctrls_rho, base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space)
    ]    
]


assigner_A.assign(A, comps_A)

In [8]:
B = Function(control_space)
assigner_B = FunctionAssigner(control_space, [base_space] * control_space.num_sub_spaces())

comps_B = [
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_lambda, base_space),
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_lambda - 2 * ctrls_mu, base_space),
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-ctrls_mu, base_space),
        interpolate(Constant(0.0), base_space)
    ],
    *[
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        project(-1. / ctrls_rho, base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space)
    ],    
    *[
        interpolate(Constant(0.0), base_space),
        project(-1. / ctrls_rho, base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space),
        interpolate(Constant(0.0), base_space)
    ],
    
]

assigner_B.assign(B, comps_B)

In [9]:
u = TrialFunction(target_space)
v = TestFunction(target_space)

In [10]:
T = 1.0 # final time
num_steps = 100 # number of time steps
dt = T / num_steps # time step size


In [11]:
# gaussian source
f = Expression(
    (
        "0.0", "0.0", "0.0",
        "exp(-a * (pow(x[0] - x_0, 2) + pow(x[1] - x_1, 2))) * exp(-b * pow(t, 2))",
        "exp(-a * (pow(x[0] - x_0, 2) + pow(x[1] - x_1, 2))) * exp(-b * pow(t, 2))"
    ), degree=2,
    x_0=5.0,
    x_1=5.0, 
    t=0., 
    a=5., 
    b=100.
)

source = interpolate(f, target_space)

In [12]:
! rm -r snapshots/
! mkdir snapshots/
! rm samples.tar

rm: cannot remove 'samples.tar': No such file or directory


In [13]:
import os

vtkfile_vx = File(
    os.path.join(os.getcwd(), 'snapshots', 'vx.pvd')) 
vtkfile_vy = File(
    os.path.join(os.getcwd(), 'snapshots', 'vy.pvd')) 

In [14]:
tol = 1e-14

class boundary_top(SubDomain):
    def inside(self, x, on_boundary): return on_boundary and near(x[1], 2000., tol)
    
class boundary_bot(SubDomain):
    def inside(self, x, on_boundary): return on_boundary and near(x[1], 0., tol)

class boundary_left(SubDomain):
    def inside(self, x, on_boundary): return on_boundary and near(x[0], 0., tol)

class boundary_right(SubDomain):
    def inside(self, x, on_boundary): return on_boundary and near(x[0], 2000., tol)
    
top   = boundary_top()
bot   = boundary_bot()
left  = boundary_left()
right = boundary_right()

# test with dirichlet boundary condition
# TODO: free boundary?

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim() - 1)
left.mark(boundaries, 1)
right.mark(boundaries, 2)
bot.mark(boundaries, 3)
top.mark(boundaries, 4)

# gaussian functor
f = Expression(
    (
        "0.0",
        "0.0",
        "0.0",
        "exp(-a * (pow(x[0] - x_0, 2) + pow(x[1] - x_1, 2))) * exp(-b * pow(t - 0.01, 2)) * sin(2. * 100. * t)",
        "exp(-a * (pow(x[0] - x_0, 2) + pow(x[1] - x_1, 2))) * exp(-b * pow(t - 0.01, 2)) * sin(2. * 100. * t)"
    ), degree=2,
    x_0=1000.0,
    x_1=1000.0, 
    t=0., 
    a=1 / 100000., 
    b=100.
)

gauss_functor = DirichletBC(
    target_space, f, top
)

free_bot_1 = DirichletBC(target_space.sub(0), Constant(0.), bot)
free_bot_2 = DirichletBC(target_space.sub(1), Constant(0.), bot)
free_bot_3 = DirichletBC(target_space.sub(2), Constant(0.), bot)

free_left_1 = DirichletBC(target_space.sub(0), Constant(0.), left)
free_left_2 = DirichletBC(target_space.sub(1), Constant(0.), left)
free_left_3 = DirichletBC(target_space.sub(2), Constant(0.), left)

free_right_1 = DirichletBC(target_space.sub(0), Constant(0.), right)
free_right_2 = DirichletBC(target_space.sub(1), Constant(0.), right)
free_right_3 = DirichletBC(target_space.sub(2), Constant(0.), right)



BC = [
    free_bot_1,
    free_bot_2,
    free_bot_3,
    free_left_1,
    free_left_2,
    free_left_3,
    free_right_1,
    free_right_2,
    free_right_3
]

u_n = interpolate(
    Expression(("0.0", "0.0", "0.0", "0.0", "0.0"), degree=2),
    target_space
)

In [15]:
%%time

u = TrialFunction(target_space)
v = TestFunction(target_space)

F = dot(u, v) * dx +\
        dt * dot((A * grad(u)[:, 0] + B * grad(u)[:, 1]), v) * dx -\
        (dot(u_n, v) + dt * dot(f, v)) * dx
    
a, L = lhs(F), rhs(F)
    
t = 0
u = Function(target_space)
    
for n in range(num_steps):
    # Update current time
    t += dt
    f.t = t
    # Compute solution
    solve(a == L, u, BC)
    u_n.assign(u)
    
    #if n % 10 == 0:        
    vtkfile_vx << (u_n.sub(3), t)
    vtkfile_vy << (u_n.sub(4), t)

CPU times: user 33.8 s, sys: 2.56 s, total: 36.3 s
Wall time: 36.3 s


In [16]:
! tar -cvf sample.tar snapshots/*

snapshots/vx.pvd
snapshots/vx000000.vtu
snapshots/vx000001.vtu
snapshots/vx000002.vtu
snapshots/vx000003.vtu
snapshots/vx000004.vtu
snapshots/vx000005.vtu
snapshots/vx000006.vtu
snapshots/vx000007.vtu
snapshots/vx000008.vtu
snapshots/vx000009.vtu
snapshots/vx000010.vtu
snapshots/vx000011.vtu
snapshots/vx000012.vtu
snapshots/vx000013.vtu
snapshots/vx000014.vtu
snapshots/vx000015.vtu
snapshots/vx000016.vtu
snapshots/vx000017.vtu
snapshots/vx000018.vtu
snapshots/vx000019.vtu
snapshots/vx000020.vtu
snapshots/vx000021.vtu
snapshots/vx000022.vtu
snapshots/vx000023.vtu
snapshots/vx000024.vtu
snapshots/vx000025.vtu
snapshots/vx000026.vtu
snapshots/vx000027.vtu
snapshots/vx000028.vtu
snapshots/vx000029.vtu
snapshots/vx000030.vtu
snapshots/vx000031.vtu
snapshots/vx000032.vtu
snapshots/vx000033.vtu
snapshots/vx000034.vtu
snapshots/vx000035.vtu
snapshots/vx000036.vtu
snapshots/vx000037.vtu
snapshots/vx000038.vtu
snapshots/vx000039.vtu
snapshots/vx000040.vtu
snapshots/vx000041.vtu
snapshots/vx00004

In [17]:
%%time

J = assemble(dot(u_n, u_n)*dx)

control = Control(A)

adj_timer = Timer("Adjoint run")
dJdm = compute_gradient(J, control) 
adj_time = adj_timer.stop()

CPU times: user 34.6 s, sys: 2.04 s, total: 36.6 s
Wall time: 36.7 s
